In [47]:
# Autoreload 
%load_ext autoreload
%autoreload 2

from pathlib import Path 
import pandas as pd 
import numpy as np
import os 
import re 
import gzip 
import shutil
import Bio.PDB.MMCIF2Dict
from typing import Union, List, Tuple, Dict, Optional
from pathlib import Path

pd.options.mode.chained_assignment = None  # default='warn'


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Get background frequencies.

In [ ]:
# TODO: 
# run compseq from within python.

In [57]:
# Load in file 
from phosphosite import DATA_DIR
from phosphosite.utils.compseq import parse_compseq_output

filepath = DATA_DIR / "compseq" / "scgr4_human.composition" 
df = parse_compseq_output(filepath)

In [58]:
df

word,AAA,AAC,AAD,AAE,AAF,AAG,AAH,AAI,AAK,AAL,...,YYP,YYQ,YYR,YYS,YYT,YYU,YYV,YYW,YYY,Other
observations,,,,,,,,,,,,,,,,,,,,,
count,13401.000000,1535.000000,2641.000000,4821.000000,2529.000000,6470.000000,1628.000000,2534.000000,3158.000000,7917.000000,...,469.000000,501.000000,719.000000,895.000000,514.000000,0.0,546.000000,164.000000,478.000000,72.000000
freq,0.001179,0.000135,0.000232,0.000424,0.000223,0.000570,0.000143,0.000223,0.000278,0.000697,...,0.000041,0.000044,0.000063,0.000079,0.000045,0.0,0.000048,0.000014,0.000042,0.000006
exp,0.000345,0.000113,0.000233,0.000349,0.000179,0.000323,0.000129,0.000213,0.000282,0.000490,...,0.000045,0.000034,0.000040,0.000059,0.000038,0.0,0.000042,0.000009,0.000019,0.000006
freq/exp,3.421967,1.193803,0.998603,1.214486,1.240463,1.761144,1.110940,1.047225,0.986669,1.422139,...,0.922630,1.305517,1.583844,1.333848,1.193247,0.0,1.137856,1.677797,2.230995,1.094863


In [50]:
total_count = df.sum(axis=1)
total_count

observations
count       1.136161e+07
freq        1.000005e+00
exp         9.999825e-01
freq/exp    2.899393e+05
dtype: float64

In [54]:
triplet_dict = {}

for triplet in df.columns: 
    triplet_dict[triplet] = df[triplet].to_dict()

triplets = list(triplet_dict.keys())

# Remove "Other" triplet
triplets.remove("Other")
len(triplets)

9261

In [27]:
20**3

8000

In [7]:
for i in [0, 1, 2]:
    amino_acids = list(set([t[i] for t in triplets]))
    aa_str = "".join(sorted(amino_acids))
    print(aa_str, "\t", len(aa_str))
    

ACDEFGHIKLMNPQRSTUVWY 	 21
ACDEFGHIKLMNPQRSTUVWY 	 21
ACDEFGHIKLMNPQRSTUVWY 	 21


In [9]:
amino_acids = aa_str
amino_acids

'ACDEFGHIKLMNPQRSTUVWY'

In [55]:
for aa_prev in amino_acids:
    for aa in amino_acids:
        for aa_next in amino_acids:
            triplet = aa_prev + aa + aa_next
            if triplet not in triplets:
                print(triplet)

In [25]:
t1 = "AAA"
t2 = "ACA"

df[[t1,t2]].sum(axis=1).freq

0.0012931

In [30]:
freq_pair_dict = {}
for aa_prev in amino_acids:
    for aa_next in amino_acids:
        # Each combination of pairs 

        # Get probability of AXA 
        triplets = [
            aa_prev + aa + aa_next
            for aa in amino_acids
        ]
        idx = aa_prev + "X" + aa_next
        freq_pair_dict[idx] = df[triplets].sum(axis=1).freq  # sum frequency for AXA 
        

In [56]:
freq_pair_dict

{'AXA': 0.006569799999999999,
 'AXC': 0.0015269,
 'AXD': 0.0030819,
 'AXE': 0.0046949,
 'AXF': 0.0022803999999999997,
 'AXG': 0.005260699999999999,
 'AXH': 0.0016816000000000003,
 'AXI': 0.0028003,
 'AXK': 0.0035503,
 'AXL': 0.007137900000000001,
 'AXM': 0.0013566,
 'AXN': 0.0022341999999999996,
 'AXP': 0.0044982,
 'AXQ': 0.0032257999999999996,
 'AXR': 0.004122600000000001,
 'AXS': 0.0057152,
 'AXT': 0.0035701,
 'AXU': 5e-07,
 'AXV': 0.004362799999999999,
 'AXW': 0.0007903,
 'AXY': 0.0017012,
 'CXA': 0.0015304,
 'CXC': 0.0007664,
 'CXD': 0.0011611,
 'CXE': 0.0017378000000000003,
 'CXF': 0.0007838000000000001,
 'CXG': 0.0013914,
 'CXH': 0.0006395,
 'CXI': 0.0009201000000000001,
 'CXK': 0.0016081999999999997,
 'CXL': 0.0020951000000000003,
 'CXM': 0.0004,
 'CXN': 0.0008845,
 'CXP': 0.0014633,
 'CXQ': 0.0011028000000000001,
 'CXR': 0.0013074000000000002,
 'CXS': 0.0018742,
 'CXT': 0.0010624,
 'CXU': 5e-07,
 'CXV': 0.0014123,
 'CXW': 0.0002782,
 'CXY': 0.0005737,
 'DXA': 0.0030424999999999

In [37]:
# Freq dict actual 
# turn df into dict with columns as keys and values as dict of amino acid frequencies
stat_dict = df.to_dict(orient="index")
freq_dict = stat_dict["freq"]
count_dict = stat_dict["count"]

In [74]:
for res in "STY":
    for aa_prev in amino_acids:
        for aa_next in amino_acids:
            # Each combination of pairs 
            triplets.append(aa_prev + res + aa_next)

metric_dict = {
    k: v 
    for k, v in freq_dict.items() 
    if k in triplets
}
len(metric_dict)
    

9261

In [68]:
from phosphosite.utils.compseq import get_frequency_pair_dict, get_triplet_dict
get_frequency_pair_dict(df, metric="freq")



{'AXA': 0.006569799999999999,
 'AXC': 0.0015269,
 'AXD': 0.0030819,
 'AXE': 0.0046949,
 'AXF': 0.0022803999999999997,
 'AXG': 0.005260599999999999,
 'AXH': 0.0016816000000000003,
 'AXI': 0.0028003,
 'AXK': 0.0035503,
 'AXL': 0.007137900000000001,
 'AXM': 0.0013566,
 'AXN': 0.0022341999999999996,
 'AXP': 0.0044982,
 'AXQ': 0.0032256999999999997,
 'AXR': 0.004122600000000001,
 'AXS': 0.0057152,
 'AXT': 0.0035701,
 'AXV': 0.004362799999999999,
 'AXW': 0.0007903,
 'AXY': 0.0017012,
 'CXA': 0.0015304,
 'CXC': 0.0007664,
 'CXD': 0.0011611,
 'CXE': 0.0017378000000000003,
 'CXF': 0.0007838000000000001,
 'CXG': 0.001391,
 'CXH': 0.0006395,
 'CXI': 0.0009201000000000001,
 'CXK': 0.0016081999999999997,
 'CXL': 0.0020951000000000003,
 'CXM': 0.0004,
 'CXN': 0.0008845,
 'CXP': 0.0014633,
 'CXQ': 0.0011028000000000001,
 'CXR': 0.0013074000000000002,
 'CXS': 0.0018742,
 'CXT': 0.0010624,
 'CXV': 0.0014123,
 'CXW': 0.0002782,
 'CXY': 0.0005737,
 'DXA': 0.003042499999999999,
 'DXC': 0.00108659999999999

In [82]:
psite_dict=get_triplet_dict(df, metric="freq", amino_acids=None, centre_residue="STY")
len(psite_dict)

1323

In [84]:
std_residues = "ACDEFGHIKLMNPQRSTVWY"
psite_dict=get_triplet_dict(df, metric="freq", amino_acids=std_residues, centre_residue="STY")
len(psite_dict), 20*3*20

(1200, 1200)